In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_pre_processing import read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from fem_pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Poisson Problem at Rectangular Domain $\Omega = [0,1]^2$

Considere um problema de _Poisson_ 2D em $\Omega = [0,1]^2$

$$
-\nabla \cdot \left( \nabla u(x,y) \right) = f(x,y)
\tag{1}
$$

onde,
$$
f(x,y) = 2\pi^2 \, sin(\pi x) \, sin(\pi y)
\tag{2}
$$

cujas condições de contorno são $u=0$ em $\partial \Omega$.

A solução analítica deste problema é

$$
u(x,y) = sin(\pi x) \, sin(\pi y)
\tag{3}
$$

Implemente um programa de elementos finitos utilizando elementos triangulares $P_1$ para resolver o Problema de Valor de Contorno $(1)$.

# 10-noded Cubic Triangular Elements, $P_3$
## Pre-processor module
## Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 3)
BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'omega'}]
MATERIAL = [{'tag': 201, 'name': 'free_space', 'a_constant': 1}]

# Create mesh from file geometry
ps.create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1E-2, view_mesh=False)

# Read mesh data
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='rectangular', info_mode=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\analytical_solution.svg
Model rectangular_domain_Triangle3 (2D)
Info     : 9 geometric entities
Info     : 2 Physical Groups
Info     : 105289 nodes in total
Info     : 23264 2-D elements in total


## Mesh Data Dictionaries

In [3]:
cell_data = mesh_data['cell']; cell_data

{1: {'Tag': 405,
  'conn': [8060, 1250, 12266, 12634, 12635, 12636, 12637, 12638, 12639, 12640],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 2: {'Tag': 406,
  'conn': [1309, 8366, 10951, 12641, 12642, 12643, 12644, 12645, 12646, 12647],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 3: {'Tag': 407,
  'conn': [7701, 1263, 12368, 12648, 12649, 12650, 12651, 12652, 12653, 12654],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 4: {'Tag': 408,
  'conn': [7688,
   10076,
   12510,
   12655,
   12656,
   12657,
   12658,
   12659,
   12660,
   12661],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 5: {'Tag': 409,
  'conn': [1361, 7715, 10245, 12662, 12663, 12664, 12665, 12666, 12667, 12668],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 6: {'Tag': 410,
  'conn': [12266,
   1250,
   12505,
   12637,
   12636,
   12669,
   12670,
   12671,
   12672,
   12673],
  'material': {'tag': 

In [4]:
nodes_data = mesh_data['nodes']; nodes_data

{1: {'xg': (0.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 2: {'xg': (1.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (1.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 4: {'xg': (0.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 5: {'xg': (0.009999999999981104, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 6: {'xg': (0.01999999999996237, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 7: {'xg': (0.02999999999994307, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 8: {'xg': (0.03999999999992186, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 9: {'xg': (0.04999999999989966, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 10: {'xg': (0.05999999999987837, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 11: {'xg': (0.06999999999986051, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 12: {'xg': (0.07999999999984518, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 13: {'xg': (0.0899999999998299, 0.0),
  'bc': {

In [5]:
conn = {key: cell['conn'] for key, cell in cell_data.items()}; conn

{1: [8060, 1250, 12266, 12634, 12635, 12636, 12637, 12638, 12639, 12640],
 2: [1309, 8366, 10951, 12641, 12642, 12643, 12644, 12645, 12646, 12647],
 3: [7701, 1263, 12368, 12648, 12649, 12650, 12651, 12652, 12653, 12654],
 4: [7688, 10076, 12510, 12655, 12656, 12657, 12658, 12659, 12660, 12661],
 5: [1361, 7715, 10245, 12662, 12663, 12664, 12665, 12666, 12667, 12668],
 6: [12266, 1250, 12505, 12637, 12636, 12669, 12670, 12671, 12672, 12673],
 7: [1263, 7701, 12506, 12649, 12648, 12674, 12675, 12676, 12677, 12678],
 8: [11060, 1278, 12372, 12679, 12680, 12681, 12682, 12683, 12684, 12685],
 9: [9082, 6446, 11559, 12686, 12687, 12688, 12689, 12690, 12691, 12692],
 10: [7759, 1349, 8921, 12693, 12694, 12695, 12696, 12697, 12698, 12699],
 11: [10951, 8366, 12504, 12644, 12643, 12700, 12701, 12702, 12703, 12704],
 12: [6984, 7646, 8988, 12705, 12706, 12707, 12708, 12709, 12710, 12711],
 13: [978, 1284, 7789, 12712, 12713, 12714, 12715, 12716, 12717, 12718],
 14: [7717, 1223, 9999, 12719, 127

## Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

In [6]:
graph_results.plot_mesh(FINITE_ELEMENT, mesh_data, model='rectangular', Numbering=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pre_processing\pictures\rectangular_meshed_domain_Triangle3.svg


<figure>
    <img src="pre_processing/pictures/rectangular_meshed_domain_Triangle3.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

## `apply_physics()`

In [7]:
mesh_data = ps.apply_physics(FINITE_ELEMENT, mesh_data); mesh_data['cell']

{1: {'Tag': 405,
  'conn': [8060, 1250, 12266, 12634, 12635, 12636, 12637, 12638, 12639, 12640],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 0.7568466042524691}},
 2: {'Tag': 406,
  'conn': [1309, 8366, 10951, 12641, 12642, 12643, 12644, 12645, 12646, 12647],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 0.742388981852404}},
 3: {'Tag': 407,
  'conn': [7701, 1263, 12368, 12648, 12649, 12650, 12651, 12652, 12653, 12654],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 0.7331255261392171}},
 4: {'Tag': 408,
  'conn': [7688,
   10076,
   12510,
   12655,
   12656,
   12657,
   12658,
   12659,
   12660,
   12661],
  'material': {

## ``global_nodes_coordinates()``

In [8]:
nodes_coord = {key: value['xg'] for key, value in nodes_data.items()}; nodes_coord

{1: (0.0, 0.0),
 2: (1.0, 0.0),
 3: (1.0, 1.0),
 4: (0.0, 1.0),
 5: (0.009999999999981104, 0.0),
 6: (0.01999999999996237, 0.0),
 7: (0.02999999999994307, 0.0),
 8: (0.03999999999992186, 0.0),
 9: (0.04999999999989966, 0.0),
 10: (0.05999999999987837, 0.0),
 11: (0.06999999999986051, 0.0),
 12: (0.07999999999984518, 0.0),
 13: (0.0899999999998299, 0.0),
 14: (0.09999999999981463, 0.0),
 15: (0.1099999999997994, 0.0),
 16: (0.1199999999997832, 0.0),
 17: (0.1299999999997609, 0.0),
 18: (0.1399999999997328, 0.0),
 19: (0.1499999999997036, 0.0),
 20: (0.1599999999996745, 0.0),
 21: (0.1699999999996453, 0.0),
 22: (0.1799999999996162, 0.0),
 23: (0.189999999999587, 0.0),
 24: (0.1999999999995579, 0.0),
 25: (0.2099999999995288, 0.0),
 26: (0.2199999999994996, 0.0),
 27: (0.2299999999994704, 0.0),
 28: (0.2399999999994413, 0.0),
 29: (0.2499999999994121, 0.0),
 30: (0.259999999999383, 0.0),
 31: (0.2699999999993538, 0.0),
 32: (0.2799999999993247, 0.0),
 33: (0.2899999999992955, 0.0),
 34: 

## $x_g, y_g$ global coordinate

In [9]:
xg = {key: value['xg'][0] for key, value in nodes_data.items()}; xg

{1: 0.0,
 2: 1.0,
 3: 1.0,
 4: 0.0,
 5: 0.009999999999981104,
 6: 0.01999999999996237,
 7: 0.02999999999994307,
 8: 0.03999999999992186,
 9: 0.04999999999989966,
 10: 0.05999999999987837,
 11: 0.06999999999986051,
 12: 0.07999999999984518,
 13: 0.0899999999998299,
 14: 0.09999999999981463,
 15: 0.1099999999997994,
 16: 0.1199999999997832,
 17: 0.1299999999997609,
 18: 0.1399999999997328,
 19: 0.1499999999997036,
 20: 0.1599999999996745,
 21: 0.1699999999996453,
 22: 0.1799999999996162,
 23: 0.189999999999587,
 24: 0.1999999999995579,
 25: 0.2099999999995288,
 26: 0.2199999999994996,
 27: 0.2299999999994704,
 28: 0.2399999999994413,
 29: 0.2499999999994121,
 30: 0.259999999999383,
 31: 0.2699999999993538,
 32: 0.2799999999993247,
 33: 0.2899999999992955,
 34: 0.2999999999992664,
 35: 0.3099999999992372,
 36: 0.3199999999992081,
 37: 0.329999999999179,
 38: 0.3399999999991498,
 39: 0.3499999999991207,
 40: 0.3599999999990915,
 41: 0.3699999999990624,
 42: 0.3799999999990332,
 43: 0.38999

## $a_e = (x_e, y_e)$ global element coordinate

In [10]:
ai = {key: [nodes_data[id]['xg'] for id in node_ids] for key, node_ids in conn.items()}
xi = {key: [coord[0] for coord in coords] for key, coords in ai.items()}
yi = {key: [coord[1] for coord in coords] for key, coords in ai.items()}
print("'a_e' global coordinates: "); ai

'a_e' global coordinates: 


{1: [(0.009051249941159936, 0.4658844834723317),
  (0.009017040531498245, 0.4560732600064985),
  (0.01882633369316201, 0.4621619222690692),
  (0.009039846804606039, 0.4626140756503873),
  (0.009028443668052142, 0.4593436678284429),
  (0.0122868049187195, 0.458102814094022),
  (0.01555656930594075, 0.4601323681815456),
  (0.01556797244249465, 0.46340277600349),
  (0.01230961119182729, 0.4646436297379108),
  (0.01229820805527339, 0.4613732219159664)],
 2: [(0.9910574949330793, 0.5643474715004013),
  (0.9909367660188946, 0.5738346780164604),
  (0.9812292488676899, 0.5678697198543028),
  (0.9910172519616843, 0.567509873672421),
  (0.9909770089902895, 0.5706722758444407),
  (0.987700926968493, 0.5718463586290745),
  (0.9844650879180915, 0.5698580392416887),
  (0.9845053308894863, 0.566695637069669),
  (0.9877814129112829, 0.5655215542850351),
  (0.9877411699398879, 0.5686839564570549)],
 3: [(0.9815399927097342, 0.4621356477470012),
  (0.9910173919499206, 0.4560136211986012),
  (0.991704513

In [11]:
print("'a_1' global coordinates: "); ai[1]

'a_1' global coordinates: 


[(0.009051249941159936, 0.4658844834723317),
 (0.009017040531498245, 0.4560732600064985),
 (0.01882633369316201, 0.4621619222690692),
 (0.009039846804606039, 0.4626140756503873),
 (0.009028443668052142, 0.4593436678284429),
 (0.0122868049187195, 0.458102814094022),
 (0.01555656930594075, 0.4601323681815456),
 (0.01556797244249465, 0.46340277600349),
 (0.01230961119182729, 0.4646436297379108),
 (0.01229820805527339, 0.4613732219159664)]

In [12]:
print("'Node 2' global coordinates: "); nodes_data[2]['xg']

'Node 2' global coordinates: 


(1.0, 0.0)

In [13]:
print("'x_1' x-coordinates: "); xi[1]

'x_1' x-coordinates: 


[0.009051249941159936,
 0.009017040531498245,
 0.01882633369316201,
 0.009039846804606039,
 0.009028443668052142,
 0.0122868049187195,
 0.01555656930594075,
 0.01556797244249465,
 0.01230961119182729,
 0.01229820805527339]

## `map_to_physical_coordinates()`

In [14]:
xi_master = (0, 1)
xg_1, yg_1 = matrices_assembly.isomapping_to_global_coordinates(ai[1], xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg_1}, {yg_1})")

'e_1' Master coordinates: (0, 1) --> Global coordinates: (0.01882633369316201, 0.4621619222690692)


## Material Proprieties, $k_a$

In [ ]:
ka = {key: value['stiffness_a_value'] for key, value in cell_data.items()}; ka

{1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 1,
 68: 1,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 79: 1,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 1,
 87: 1,
 88: 1,
 89: 1,
 90: 1,
 91: 1,
 92: 1,
 93: 1,
 94: 1,
 95: 1,
 96: 1,
 97: 1,
 98: 1,
 99: 1,
 100: 1,
 101: 1,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 1,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 1,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 123: 1,
 

## Source: Analytical source, $f(x)$

In [16]:
fx = {key: value['source']['value'] for key, value in cell_data.items()}; fx

{1: 0.7568466042524691,
 2: 0.742388981852404,
 3: 0.7331255261392171,
 4: 0.8467831646010973,
 5: 0.5802606357072025,
 6: 0.9131043260941691,
 7: 0.9046611162833667,
 8: 0.5694681343685275,
 9: 1.3430490151437169,
 10: 0.876471280444023,
 11: 0.8973178104435577,
 12: 1.2505512172020707,
 13: 0.25166380830746415,
 14: 0.5668747084628206,
 15: 0.6056501112723313,
 16: 0.5176086819842384,
 17: 1.4204785053978124,
 18: 0.3921550936160858,
 19: 0.712481423118198,
 20: 0.7106256922289887,
 21: 0.7480171965838586,
 22: 0.7545624922965015,
 23: 0.7587101036760657,
 24: 0.3764879810858784,
 25: 0.2282562245259882,
 26: 0.6637472796438196,
 27: 1.4526603419459474,
 28: 0.7321880655078874,
 29: 0.6337916870470385,
 30: 0.2547336601203856,
 31: 0.5639546952923769,
 32: 0.378958392402643,
 33: 0.6795567816995061,
 34: 0.341304095203282,
 35: 0.6127673609172644,
 36: 0.27517411187270957,
 37: 0.6172286455246564,
 38: 0.6195757859728347,
 39: 0.711049582715056,
 40: 0.6970599878077354,
 41: 0.693181

## Jacobian Matrix Transform

In [17]:
Je = matrices_assembly.jacobian(FINITE_ELEMENT, mesh_data, e=1, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_1:\n", Je)
print("Determinant of the Jacobian matrix for element e_1:", Jdet)

Jacobian matrix for element e_1:
 [[-3.42094097e-05 -9.81122347e-03]
 [ 9.77508375e-03 -3.72256120e-03]]
Determinant of the Jacobian matrix for element e_1: 9.60328777093225e-05


## Local Elements $e_1$

In [18]:
Ae, fe, Me, area_e = matrices_assembly.local_matrices(FINITE_ELEMENT, mesh_data, e=1)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", fe)
print("Area of element e_1: ", area_e)

Local stiffness matrix for element e_1:
 [[ 5.89901763e-01+0.j -6.67152691e-02+0.j -5.47351087e-02+0.j
  -4.91202677e-01+0.j  2.80788208e-01+0.j -5.20501542e-02+0.j
  -5.20501581e-02+0.j  2.39713378e-01+0.j -3.93649954e-01+0.j
  -2.76778383e-08+0.j]
 [-6.67152691e-02+0.j  4.84200159e-01+0.j -3.29729973e-02+0.j
   2.71461615e-01+0.j -5.00529367e-01+0.j -2.25770871e-01+0.j
   1.55773816e-01+0.j -4.27235381e-02+0.j -4.27235331e-02+0.j
  -1.34257882e-08+0.j]
 [-5.47351087e-02+0.j -3.29729973e-02+0.j  4.26010809e-01+0.j
  -3.75891801e-02+0.j -3.75891847e-02+0.j  1.50639463e-01+0.j
  -2.30905228e-01+0.j -4.08110987e-01+0.j  2.25252423e-01+0.j
  -9.90584931e-09+0.j]
 [-4.91202677e-01+0.j  2.71461615e-01+0.j -3.75891801e-02+0.j
   2.97816493e+00+0.j -1.36762395e+00+0.j  2.11121141e-01+0.j
   2.11121090e-01+0.j  1.27181555e-01+0.j -6.35907842e-01+0.j
  -1.26672668e+00+0.j]
 [ 2.80788208e-01+0.j -5.00529367e-01+0.j -3.75891847e-02+0.j
  -1.36762395e+00+0.j  2.97816503e+00+0.j -1.05560562e+00+0.j

## Global “stiffness” matrix, $A_g$

In [19]:
Ag, fg, Mg = matrices_assembly.global_matrix(FINITE_ELEMENT, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (105289, 105289)


## Imposition of Boundary Conditions
### $Dirichlet$ nodes

In [20]:
dirichlet_nodes = {key: value 
                   for key, value in nodes_data.items() if value['bc']['type'] == 'Dirichlet'}
dirichlet_nodes

{1: {'xg': (0.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 2: {'xg': (1.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (1.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 4: {'xg': (0.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 5: {'xg': (0.009999999999981104, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 6: {'xg': (0.01999999999996237, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 7: {'xg': (0.02999999999994307, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 8: {'xg': (0.03999999999992186, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 9: {'xg': (0.04999999999989966, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 10: {'xg': (0.05999999999987837, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 11: {'xg': (0.06999999999986051, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 12: {'xg': (0.07999999999984518, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 13: {'xg': (0.0899999999998299, 0.0),
  'bc': {

In [21]:
Nn = len(nodes_data); Nd = len(dirichlet_nodes)
print(f"The entire domain has {Nn} nodes: {Nn - Nd} free nodes; {Nd} Dirichlet nodes.")

The entire domain has 105289 nodes: 104089 free nodes; 1200 Dirichlet nodes.


In [22]:
free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}; free_nodes

{1201: {'xg': (0.4950000000020247, 0.9913397459621618),
  'bc': {'type': 'Free', 'value': None}},
 1202: {'xg': (0.009211990098867707, 0.4961881723756173),
  'bc': {'type': 'Free', 'value': None}},
 1203: {'xg': (0.504999999998702, 0.008660254037843167),
  'bc': {'type': 'Free', 'value': None}},
 1204: {'xg': (0.990998005876526, 0.504174558101478),
  'bc': {'type': 'Free', 'value': None}},
 1205: {'xg': (0.00880561502776083, 0.5847550863993457),
  'bc': {'type': 'Free', 'value': None}},
 1206: {'xg': (0.5850000000017108, 0.9913397459623841),
  'bc': {'type': 'Free', 'value': None}},
 1207: {'xg': (0.4149999999989312, 0.00866025403781913),
  'bc': {'type': 'Free', 'value': None}},
 1208: {'xg': (0.9913397459621809, 0.4149999999989312),
  'bc': {'type': 'Free', 'value': None}},
 1209: {'xg': (0.4050000000016802, 0.9913397459621234),
  'bc': {'type': 'Free', 'value': None}},
 1210: {'xg': (0.9915083687974559, 0.59529206331703),
  'bc': {'type': 'Free', 'value': None}},
 1211: {'xg': (0.00

## Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [23]:
{global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

{1201: 1,
 1202: 2,
 1203: 3,
 1204: 4,
 1205: 5,
 1206: 6,
 1207: 7,
 1208: 8,
 1209: 9,
 1210: 10,
 1211: 11,
 1212: 12,
 1213: 13,
 1214: 14,
 1215: 15,
 1216: 16,
 1217: 17,
 1218: 18,
 1219: 19,
 1220: 20,
 1221: 21,
 1222: 22,
 1223: 23,
 1224: 24,
 1225: 25,
 1226: 26,
 1227: 27,
 1228: 28,
 1229: 29,
 1230: 30,
 1231: 31,
 1232: 32,
 1233: 33,
 1234: 34,
 1235: 35,
 1236: 36,
 1237: 37,
 1238: 38,
 1239: 39,
 1240: 40,
 1241: 41,
 1242: 42,
 1243: 43,
 1244: 44,
 1245: 45,
 1246: 46,
 1247: 47,
 1248: 48,
 1249: 49,
 1250: 50,
 1251: 51,
 1252: 52,
 1253: 53,
 1254: 54,
 1255: 55,
 1256: 56,
 1257: 57,
 1258: 58,
 1259: 59,
 1260: 60,
 1261: 61,
 1262: 62,
 1263: 63,
 1264: 64,
 1265: 65,
 1266: 66,
 1267: 67,
 1268: 68,
 1269: 69,
 1270: 70,
 1271: 71,
 1272: 72,
 1273: 73,
 1274: 74,
 1275: 75,
 1276: 76,
 1277: 77,
 1278: 78,
 1279: 79,
 1280: 80,
 1281: 81,
 1282: 82,
 1283: 83,
 1284: 84,
 1285: 85,
 1286: 86,
 1287: 87,
 1288: 88,
 1289: 89,
 1290: 90,
 1291: 91,
 1292: 9

## Processor Module
## Asymmetric Global matrix $A_g$ with boundary conditions

In [24]:
Ag, fg = boundary_conditions.apply_simple_dirichlet(Ag, fg, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (105289, 105289)


## Global Potential Vector

In [25]:
# Make sure Ag is in CSR format for efficiency
# Convert fg to a dense format or keep it sparse as needed
# Solve the linear system Ag * u = fg
ug = spsolve(Ag.tocsr(), fg.toarray())
print("The solution vector u is: \n", ug)

The solution vector u is: 
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.02049832+0.j
 0.01668215+0.j 0.01527332+0.j]


## Reduced Global matrix $A_{gr}$ with boundary conditions

In [26]:
Agr, fgr, Mgr = boundary_conditions.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (104089, 104089)


## Global Potential Solution

In [27]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.global_potentials_solution(mesh_data, ur)
print("Reduced solution vector ur: ", ur)
print("Global potential vector V: "); potential_u

Reduced solution vector ur:  [0.02719916+0.j 0.0289331 +0.j 0.02719915+0.j ... 0.02049832+0.j
 0.01668215+0.j 0.01527332+0.j]
Global potential vector V: 


{1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.0,
 100: 0.0,
 101: 0.

## Solution at each element

In [28]:
u_for_cell = {}
for en, cell in mesh_data['cell'].items():
    u_e = []
    for node in cell['conn']:
        u_e.append(potential_u[node])
    u_for_cell[en] = u_e

print("The complete solution for each cell."); u_for_cell

The complete solution for each cell.


{1: [(0.028266992627274577+0j),
  (0.02805348702010984+0j),
  (0.05869057297608926+0j),
  (0.028198755957015383+0j),
  (0.028127620261510576+0j),
  (0.03825525403265857+0j),
  (0.04846770437511616+0j),
  (0.0485633892099036+0j),
  (0.038422318070046466+0j),
  (0.038340539314289906+0j)],
 2: [(0.027516692427391066+0j),
  (0.027705364593635463+0j),
  (0.05759883646738542+0j),
  (0.02758289149205024+0j),
  (0.02764581630587184+0j),
  (0.037647757526965975+0j),
  (0.04761232304254053+0j),
  (0.04759197869803345+0j),
  (0.03756472810704472+0j),
  (0.03760815639768198+0j)],
 3: [(0.05754953992973009+0j),
  (0.02794573104166668+0j),
  (0.025907695461175908+0j),
  (0.04767117905736236+0j),
  (0.03780316132097246+0j),
  (0.027271189712243064+0j),
  (0.026591790315695606+0j),
  (0.036469690946527106+0j),
  (0.047016843914789566+0j),
  (0.03713909449352065+0j)],
 4: [(0.05590030512866392+0j),
  (0.026405547033868312+0j),
  (0.04642552199655994+0j),
  (0.04606530983988992+0j),
  (0.036233939293863

## Interpolate Solution at element, $e_n$

In [29]:
xi_master = (0, 0.5)
en = 1
u_at_node = matrices_assembly.isomapping_to_global_coordinates(u_for_cell[en], xi_master, FINITE_ELEMENT)

print(f"Element e_{en}: Potential 'u' interpolate at xi = {xi_master} --> u(xi): {u_at_node}")

Element e_1: Potential 'u' interpolate at xi = (0, 0.5) --> u(xi): [0.04349461+0.j]


## Post-Processor

In [30]:
graph_results.fem_solution(FINITE_ELEMENT, mesh_data, potential_u, type='real')
graph_results.fem_griddata(FINITE_ELEMENT, mesh_data, potential_u, type='real')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\fem_solution_Triangle3.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\fem_solution_griddata_Triangle3.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/fem_solution_Triangle3.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### Figure 3: Scalar Field Distribution
<figure>
    <img src="pos_processing/pictures/fem_solution_griddata_Triangle3.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 3: Post-Processor: Scalar Field Distribution and Triangular Mesh.</figcaption>
</figure>

### Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/analytical_solution.svg" alt="Fig.4" style="width:100%;" />
    <figcaption>Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.